In [ ]:
import os
import glob
import pickle
import pandas as pd

The code below opens different windows_size pkl files and gets all strategies in a single dict

In [ ]:

def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def calculate_probability_of_returns(returns, threshold=1):
    return sum(r > threshold for r in returns) / len(returns)

directory_path = '..\\pkl'
window_sizes = [30, 60, 90, 120, 150, 180]
pattern = '*.pkl'

all_smart_strategies = {ws: [] for ws in window_sizes}
all_dummy_strategies = {ws: [] for ws in window_sizes}

for window_size in window_sizes:
    pickle_files = glob.glob(os.path.join(directory_path, f'*{window_size}*.pkl'))
    
    for pickle_file in pickle_files:
        backtest_portfolio = load_pickle(pickle_file)
        all_smart_strategies[window_size].extend(backtest_portfolio.smart_strategies)
        all_dummy_strategies[window_size].extend(backtest_portfolio.dummy_strategies)


Change to all_smart_strategies or all_dummy_strategies

In [ ]:
probabilities_dict = {}

for key, strategies in all_smart_strategies.items():
    metrics = {} 
    metric_count = {}
    
    for strategy in strategies:
        for metric in strategy.keys():
            if metric not in metrics:
                metrics[metric] = 0
                metric_count[metric] = 0

    for strategy in strategies:
        for metric, value in strategy.items():
            if value > 1:
                metrics[metric] += 1
            metric_count[metric] += 1
    
    probabilities = {metric: (metrics[metric] / metric_count[metric] if metric_count[metric] > 0 else 0)
                     for metric in metrics}
    
    probabilities_dict[key] = probabilities


df_probabilities = pd.DataFrame(probabilities_dict).T